# Download Data and Preprocess Data and Upload to kaggle dataset

1. In this notebook you are going to download data from zindi 
2. Extract all the .zip files 
3. Convert all train & test .wav files to 32k sample_rate
4. Upload dataset to kaggle for easy download for every notebook 

### 1. Download data from zindi

In [ ]:
%%time
import requests, os
import requests, zipfile

os.makedirs("input")

#the url and auth_value from the website 
url = 'https://api.zindi.africa/v1/competitions/giz-nlp-agricultural-keyword-spotter/files/Train.csv'
myobj = {'auth_token': '############################'} #use your own

x = requests.post(url, data = myobj,stream=True)
target_path = 'input/Train.csv'

handle = open(target_path, "wb")
for chunk in x.iter_content(chunk_size=512):
    if chunk:  # filter out keep-alive new chunks
        handle.write(chunk)
handle.close()

#the url and auth_value from the website 
url = 'https://api.zindi.africa/v1/competitions/giz-nlp-agricultural-keyword-spotter/files/SampleSubmission.csv'
myobj = {'auth_token': '############################'} #use your own

x = requests.post(url, data = myobj,stream=True)
target_path = 'input/SampleSubmission.csv'

handle = open(target_path, "wb")
for chunk in x.iter_content(chunk_size=512):
    if chunk:  # filter out keep-alive new chunks
        handle.write(chunk)
handle.close()

#the url and auth_value from the website 
url = 'https://api.zindi.africa/v1/competitions/giz-nlp-agricultural-keyword-spotter/files/audio_files.zip'
myobj = {'auth_token': '###########################'} #use your own

x = requests.post(url, data = myobj,stream=True)
target_path = 'input/audio_files.zip'

handle = open(target_path, "wb")
for chunk in x.iter_content(chunk_size=512):
    if chunk:  # filter out keep-alive new chunks
        handle.write(chunk)
handle.close()

#the url and auth_value from the website 
url = 'https://api.zindi.africa/v1/competitions/giz-nlp-agricultural-keyword-spotter/files/AdditionalUtterances.zip'
myobj = {'auth_token': '##########################'} #use your own

x = requests.post(url, data = myobj,stream=True)
target_path = 'input/AdditionalUtterances.zip'

handle = open(target_path, "wb")
for chunk in x.iter_content(chunk_size=512):
    if chunk:  # filter out keep-alive new chunks
        handle.write(chunk)
handle.close()

#the url and auth_value from the website 
url = 'https://api.zindi.africa/v1/competitions/giz-nlp-agricultural-keyword-spotter/files/nlp_keywords_29Oct2020.zip'
myobj = {'auth_token': '###########################'} #use your own

x = requests.post(url, data = myobj,stream=True)
target_path = 'input/nlp_keywords_29Oct2020.zip'

handle = open(target_path, "wb")
for chunk in x.iter_content(chunk_size=512):
    if chunk:  # filter out keep-alive new chunks
        handle.write(chunk)
handle.close()

### 2.Extract all zipfiles

In [ ]:
!unzip -q "input/audio_files.zip" -d "input"
!unzip -q "input/AdditionalUtterances.zip" -d "input"
!unzip -q "input/nlp_keywords_29Oct2020.zip" -d "input"

### 3.convert audio files to 32k sample rate

In [ ]:
!pip -q install soundfile

In [ ]:
import os, random, glob
import numpy as np, pandas as pd
import librosa
import soundfile as sf
from tqdm.auto import tqdm

In [ ]:
train = pd.read_csv("input/Train.csv")
sample_submission = pd.read_csv("input/SampleSubmission.csv")

In [ ]:
# move all training data to "input/audio_train"
# move all test data to "input/audio_test"
os.makedirs("input/audio_train", exist_ok=True)
os.makedirs("input/audio_test", exist_ok=True)

In [ ]:
# convert all train data to 32k sample rate
target_sr = 32000
for f, label in tqdm(zip(train.fn.values, train.label.values), total=len(train.fn.values)):
    os.makedirs(f"input/audio_train/{label}", exist_ok=True)
    f_name = f.split("/")[-1]
    y, sr = librosa.load(f"input/{f}", sr=target_sr)
    sf.write(f"input/audio_train/{label}/{f_name}", y, samplerate=target_sr)

In [ ]:
# convert all test data to 32k sample rate
target_sr = 32000
for f in tqdm(sample_submission.fn.values):
    f_name = f.split("/")[-1]
    y, sr = librosa.load(f"input/{f}", sr=target_sr)
    sf.write(f"input/audio_test/{f_name}", y, samplerate=target_sr)

In [ ]:
## latest_keywords
latest_keywords = glob.glob("input/latest_keywords/*/*.wav")
for f in tqdm(latest_keywords):
    label = f.split("/")[-2]
    f_name = f.split("/")[-1]
    os.makedirs(f"input/audio_train/{label}", exist_ok=True)
    y, sr = librosa.load(f, sr=target_sr)
    sf.write(f"input/audio_train/{label}/{f_name}", y, samplerate=target_sr)

In [ ]:
## nlp_keywords
nlp_keywords = glob.glob("input/nlp_keywords/*/*.wav")
for f in tqdm(nlp_keywords):
    label = f.split("/")[-2]
    f_name = f.split("/")[-1]
    os.makedirs(f"input/audio_train/{label}", exist_ok=True)
    y, sr = librosa.load(f, sr=target_sr)
    sf.write(f"input/audio_train/{label}/{f_name}", y, samplerate=target_sr)

In [ ]:
train_wav = glob.glob("input/audio_train/*/*.wav")
len(train_wav)

4709

In [ ]:
# create train dataframe
train_df = pd.DataFrame({
    "fn" : train_wav,
})
train_df["label"] = train_df.fn.apply(lambda x: x.split("/")[-2])

In [ ]:
train_df.head()

,fn,label
0,input/audio_train/amalagala/1b5a69c4b88b41eb95...,amalagala
1,input/audio_train/amalagala/1b5a69c4b88b41eb95...,amalagala
2,input/audio_train/amalagala/6fbfcce2cf21419e97...,amalagala
3,input/audio_train/amalagala/NN1FW91.wav,amalagala
4,input/audio_train/amalagala/FUUQEXZ.wav,amalagala


### 4.Upload those files to kaggle dataset so its easy to download every time

- for uploading to kaggle datasets we need to have kaggle account 
- for more detalils https://www.kaggle.com/docs/api
- i made this dataset to public after end of this compitation

In [ ]:
# make sure your kaggle api "kaggle.json" file in your drive
! mkdir /root/.kaggle
! cp '/content/drive/My Drive/kaggle.json' /root/.kaggle # <---- path for kaggle.json file
! chmod 400 /root/.kaggle/kaggle.json

!pip uninstall -y kaggle >> quit
!pip install --upgrade pip >> quit
!pip install kaggle==1.5.6 >> quit
!kaggle -v >> quit

In [ ]:
!mkdir "dataset"

In [ ]:
!zip -r "audio_train.zip" "input/audio_train" >> quit

In [ ]:
!zip -r "dataset/audio_test.zip" "input/audio_test" >> quit

In [ ]:
!cp "input/SampleSubmission.csv" "dataset"

In [ ]:
data = '''{
  "title": "giz-nlp-agricultural-keyword-spotter",
  "id": "gopidurgaprasad/giz-nlp-agricultural-keyword-spotter",
  "licenses": [
    {
      "name": "CC0-1.0"
    }
  ]
}
'''
text_file = open("dataset/dataset-metadata.json", 'w+')
n = text_file.write(data)
text_file.close()

In [ ]:
!kaggle datasets create -p "dataset"

Starting upload for file audio_test.zip
100% 104M/104M [00:12<00:00, 8.56MB/s] 
Upload successful: audio_test.zip (104MB)
Starting upload for file audio_train.zip
100% 474M/474M [00:32<00:00, 15.4MB/s]
Upload successful: audio_train.zip (474MB)
Starting upload for file SampleSubmission.csv
100% 409k/409k [00:04<00:00, 86.2kB/s]
Upload successful: SampleSubmission.csv (409KB)
Your private Dataset is being created. Please check progress at https://www.kaggle.com/gopidurgaprasad/giz-nlp-agricultural-keyword-spotter
